In [1]:
## This is demo for kuka reaching a with mpc and diff_qp
## Author : Avadesh Meduri
## Date : 25/02/2022

import time
import numpy as np
import pinocchio as pin
from robot_properties_kuka.config import IiwaConfig

import meshcat
import meshcat.transformations as tf
import meshcat.geometry as g

from diff_pin_costs import DiffFK
from inverse_qp import IOC

import torch
from torch.autograd import Function
from torch.nn import functional as F

In [2]:
robot = IiwaConfig.buildRobotWrapper()
model, data = robot.model, robot.data
f_id = model.getFrameId("EE")

/opt/openrobots/lib/python3.8/site-packages/pinocchio/shortcuts.py:45: UserWarning: You passed package dir(s) via argument geometry_model and provided package_dirs.
  geom_model = pin.buildGeomFromUrdf(model, filename, geometry_type, package_dirs)


In [3]:
viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(open=False)
viz.loadViewerModel()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [4]:
dfk = DiffFK.apply
def quadratic_loss(q_pred, x_des, nq):
    loss = torch.linalg.norm(dfk(q_pred[-2*nq:], model, data, f_id) - x_des)
    loss += 0.2*torch.linalg.norm(q_pred[-nq:])
    return loss

# def running_quadratic_loss(q_pred, x_des, nq, n_col):
#     loss = 0
#     for i in range(0, n_col):
#         loss += torch.linalg.norm(dfk(q_pred[3*i*nq:(3*i+2)*nq], model, data, f_id) - x_des)
#     return loss

In [5]:
nq = model.nq
nv = model.nv

n_col = 10
u_max = model.nq*[5,]

lr = 1e-1
eps = 100

# q_des = np.array([-0.58904862, -0.58904862, -0.58904862,  0.58904862,  0.        ,
#         0.        ,  0.        ])
q_des = np.hstack(((np.pi/3)*(np.random.rand(5) - 0.5)*2, np.zeros(2)))

dq_des = np.zeros_like(q_des)
pin.forwardKinematics(model, data, q_des, dq_des, np.zeros(nv))
pin.updateFramePlacements(model, data)

x_des = torch.tensor(data.oMf[f_id].translation)
print(x_des)
x_init = np.zeros(2*nq)
x_init[0:nq] = (np.pi/3.0)*(np.random.rand(len(q_des)) - 0.5)*2
x_init[nq:] = 0.2*2*(np.random.rand(len(q_des)) - 0.5)



tensor([-0.0759,  0.1103,  1.1709], dtype=torch.float64)


In [6]:
x_in = x_init
print(x_in)

viz.viewer["box"].set_object(g.Sphere(0.05), 
                         g.MeshLambertMaterial(
                             color=0xff22dd,
                             reflectivity=0.8))
viz.viewer["box"].set_transform(tf.translation_matrix(x_des.detach().numpy()))

for j in range(8):

    ioc = IOC(n_col, nq, u_max, 0.05, eps = 1.0, isvec=False)
    optimizer = torch.optim.Adam(ioc.parameters(), lr=lr)
    
    
    i = 0
    loss = 1000

    while loss > 0.03 and i < eps:
        x_pred = ioc(x_in) 
        loss = quadratic_loss(x_pred, x_des, nq)
        print("Index :" + str(i) + " loss is : " + str(loss.detach().numpy()), end = '\r', flush = True)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i += 1
        
    x_pred = ioc(x_in).detach().numpy()

    for i in range(n_col+1):
        q = x_pred[3*nq*i:3*nq*i + nq]
        dq = x_pred[3*nq*i + nq:3*nq*i + 2*nq]

        pin.forwardKinematics(model, data, q, dq, np.zeros(nv))
        pin.updateFramePlacements(model, data)

        viz.display(q)
        time.sleep(0.05)
    
    x_in = torch.tensor(x_pred[-2*nq:])


[-0.79073911  0.15092709  0.29922812 -0.15536367 -0.57496592  0.46207829
  0.51784889  0.14581096 -0.09285531  0.15168797 -0.08222596 -0.1779172
  0.15361998 -0.00437347]


KeyboardInterrupt: 